In [25]:
import requests
import base64
import datetime
from urllib.parse import urlencode

In [26]:
client_id = '**'
client_secret = '**'

In [27]:
class SpotifyAPI(object):
    access_token = None
    access_token_expires = datetime.datetime.now()
    access_token_did_expire = True
    client_id = None
    client_secret = None
    token_url = "https://accounts.spotify.com/api/token"

    def __init__(self, client_id, client_secret, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.client_id = client_id
        self.client_secret = client_secret
    
    def get_client_credentials(self):
        """
        Return a base64 encoded string
        """
        client_id = self.client_id
        client_secret = self.client_secret
        if client_secret == None or client_id == None:
            raise Exception("You must set client_id and client_secret")
        client_creds = f'{client_id}:{client_secret}'
        client_creds_b64 = base64.b64encode(client_creds.encode())
        return client_creds_b64.decode()

    def get_token_headers(self):
        client_creds_b64 = self.get_client_credentials()
        return {
            'Authorization': f'Basic {client_creds_b64}'
            }

    def get_token_data(self):
        return {
            'grant_type': 'client_credentials'
            }   

    def perform_authentication(self):
        token_url = self.token_url
        token_data = self.get_token_data()
        token_header = self.get_token_headers()
        r = requests.post(token_url, data=token_data, headers=token_header)
        #print(r.json())
        if r.status_code not in range(200, 299):
            raise Exception("Could not authentication client")
            # return False
        data = r.json()
        now = datetime.datetime.now()
        access_token = data['access_token']
        expires_in = data['expires_in'] # seconds
        expires = now + datetime.timedelta(seconds=expires_in)
        self.access_token = access_token
        self.access_token_expires = expires
        self.access_token_did_expire = expires < now
        return True

    def get_access_token(self):
        token = self.access_token
        expires = self.access_token_expires
        now = datetime.datetime.now()
        if expires < now :
            self.perform_authentication()
            return self.get_access_token()
        elif token == None:
            self.perform_authentication()
            return self.get_access_token()
        return token

    def get_resource_header(self):
        access_token = self.get_access_token()
        headers = {
            'Authorization': f'Bearer {access_token}'
        }
        return headers

    def get_resource(self, lookup_id, resource_type='albums', version='v1'):
        endpoint = f'https://api.spotify.com/{version}/{resource_type}/{lookup_id}'
        headers = self.get_resource_header()
        r = requests.get(endpoint, headers=headers)
        if r.status_code not in range(200, 299):
            return {}
        return r.json()

    def get_album(self, _id):
        return self.get_resource(_id, resource_type='albums')

    def get_artist(self, _id):
        return self.get_resource(_id, resource_type='artists')

    def base_search(self, query_params):
        headers = self.get_resource_header()
        endpoint = 'https://api.spotify.com/v1/search'
        lookup_url = f'{endpoint}?{query_params}'
        #print(lookup_url)
        r = requests.get(lookup_url, headers=headers)
        #print(r.status_code)
        if r.status_code not in range(200, 299):
            return {}
        return r.json()

    def search(self, query=None,operator=None, operator_query=None, search_type='artist'):
        if query is None:
            raise Exception("A query is required")
        if isinstance(query, dict):
            query = " ".join([f"{k}:{v}" for k, v in query.items()])
        if operator != None and operator_query !=None:
            if operator == "or" or operator.lower() == "not":
                operator = operator.upper()
                if isinstance(operator_query, str):
                    query = f"{query} {operator} {operator_query}"
        query_params = urlencode({'q': query, 'type': search_type.lower()})
        print(query_params)
        return self.base_search(query_params)


In [28]:
spotify = SpotifyAPI(client_id, client_secret)

In [29]:
#spotify.perform_authentication()
spotify.search({"track": "habibi", "artist": "Ra"},search_type="track")

q=track%3Ahabibi+artist%3ARa&type=track


{'tracks': {'href': 'https://api.spotify.com/v1/search?query=track%3Ahabibi+artist%3ARa&type=track&offset=0&limit=20',
  'items': [{'album': {'album_type': 'single',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/4CflzQprp6nZxKiv0t78tH'},
       'href': 'https://api.spotify.com/v1/artists/4CflzQprp6nZxKiv0t78tH',
       'id': '4CflzQprp6nZxKiv0t78tH',
       'name': 'Mohamed Ramadan',
       'type': 'artist',
       'uri': 'spotify:artist:4CflzQprp6nZxKiv0t78tH'}],
     'available_markets': ['AD',
      'AE',
      'AG',
      'AL',
      'AM',
      'AO',
      'AR',
      'AT',
      'AU',
      'AZ',
      'BA',
      'BB',
      'BD',
      'BE',
      'BF',
      'BG',
      'BH',
      'BI',
      'BJ',
      'BN',
      'BO',
      'BR',
      'BS',
      'BT',
      'BW',
      'BY',
      'BZ',
      'CA',
      'CD',
      'CG',
      'CH',
      'CI',
      'CL',
      'CM',
      'CO',
      'CR',
      'CV',
      'CW',
      'CY',
      'C

In [30]:
# spotify.get_artist('4CflzQprp6nZxKiv0t78tH')

In [31]:
# spotify.get_album("41zMFsCjcGenYKVJYUXU2n")

In [32]:
spotify.search(query='habibi', operator='NOT', operator_query='Mohamed Ramadan',search_type="track")

q=habibi+NOT+Mohamed+Ramadan&type=track


{'tracks': {'href': 'https://api.spotify.com/v1/search?query=habibi+NOT+Mohamed+Ramadan&type=track&offset=0&limit=20',
  'items': [{'album': {'album_type': 'compilation',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/3TQIvMCVNOLxytHD4FWm9a'},
       'href': 'https://api.spotify.com/v1/artists/3TQIvMCVNOLxytHD4FWm9a',
       'id': '3TQIvMCVNOLxytHD4FWm9a',
       'name': 'Saad Ramadan',
       'type': 'artist',
       'uri': 'spotify:artist:3TQIvMCVNOLxytHD4FWm9a'}],
     'available_markets': ['AD',
      'AE',
      'AG',
      'AL',
      'AM',
      'AO',
      'AR',
      'AT',
      'AU',
      'AZ',
      'BA',
      'BB',
      'BD',
      'BE',
      'BF',
      'BG',
      'BH',
      'BI',
      'BJ',
      'BN',
      'BO',
      'BR',
      'BS',
      'BT',
      'BW',
      'BY',
      'BZ',
      'CA',
      'CD',
      'CG',
      'CH',
      'CI',
      'CL',
      'CM',
      'CO',
      'CR',
      'CV',
      'CW',
      'CY',
      